## Optimización de Hiperparámetros y Selección de Modelos

Los **hiperparámetros** son configuraciones que se definen antes de entrenar un modelo de Machine Learning. A diferencia de los parámetros del modelo (que se aprenden durante el entrenamiento), los hiperparámetros no se aprenden y deben ser ajustados manualmente.

- Profundidad máxima de un árbol de decisión.

- Número de árboles en un Random Forest.

- Tasa de aprendizaje en un modelo de Gradient Boosting.

### Por qué es importante optimizar los hiperparámetros?
La elección de los hiperparámetros puede tener un impacto significativo en el rendimiento del modelo. Unos hiperparámetros mal elegidos pueden llevar a:

- **Sobreajuste (overfitting)**: El modelo se ajusta demasiado a los datos de entrenamiento y no generaliza bien a nuevos datos.

- **Subajuste (underfitting)**: El modelo es demasiado simple y no captura los patrones subyacentes en los datos.

### Técnicas para la Optimización de Hiperparámetros

#### **1.** **Grid Search** (Búsqueda en Rejilla)

- **Grid Search** es una técnica que prueba todas las combinaciones posibles de hiperparámetros en una "rejilla" predefinida.

- Por ejemplo, si tienes dos hiperparámetros con 3 valores posibles cada uno, Grid Search probará 9 combinaciones.

**Ventajas**:

- Es exhaustivo: prueba todas las combinaciones posibles.

- Fácil de implementar.

**Desventajas**:

- Computacionalmente costoso, especialmente con muchos hiperparámetros o valores posibles.

#### **2.** **Cross-Validation** (Validación Cruzada)

- **Cross-Validation** es una técnica para evaluar el rendimiento del modelo de manera robusta.

- Divide los datos en k particiones (folds), entrena el modelo en k-1 particiones y lo valida en la partición restante. Este proceso se repite k veces, y se promedian los resultados.

**Ventajas**:

- Proporciona una estimación más confiable del rendimiento del modelo.

- Ayuda a detectar sobreajuste.

**Desventajas**:

- Computacionalmente más costoso que una sola división entrenamiento/prueba.

### Uso de CrossValidator y ParamGridBuilder en Spark MLlib
Spark MLlib proporciona herramientas integradas para realizar Grid Search y Cross-Validation de manera eficiente en grandes conjuntos de datos distribuidos.

#### **1. ParamGridBuilder**

- Permite definir una "rejilla" de hiperparámetros para probar.

- Por ejemplo, puedes especificar diferentes valores para la profundidad máxima de un árbol o el número de árboles en un Random Forest.

**Ejemplo**:

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder

val paramGrid = new ParamGridBuilder()
  .addGrid(rf.maxDepth, Array(5, 10, 15)) // Profundidad máxima del árbol
  .addGrid(rf.numTrees, Array(10, 50, 100)) // Número de árboles
  .build()

#### **2. CrossValidator**

- Combina Grid Search con Cross-Validation.

- Entrena y evalúa el modelo para cada combinación de hiperparámetros en la rejilla, utilizando validación cruzada.

- Selecciona la combinación de hiperparámetros que produce el mejor rendimiento.

**Ejemplo**:

In [ ]:
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.ml.evaluation.RegressionEvaluator

val evaluator = new RegressionEvaluator()
  .setLabelCol("quality")
  .setPredictionCol("prediction")
  .setMetricName("rmse")

val cv = new CrossValidator()
  .setEstimator(pipeline) // Pipeline con el modelo
  .setEvaluator(evaluator) // Métrica de evaluación
  .setEstimatorParamMaps(paramGrid) // Rejilla de hiperparámetros
  .setNumFolds(5) // Número de folds en la validación cruzada
  .setParallelism(2) // Número de hilos para ejecución en paralelo

val cvModel = cv.fit(trainingData) // Entrenar el modelo con Cross-Validation

#### Flujo de Trabajo en Spark MLlib  

**1. Definir el Pipeline**:

- Incluye todas las transformaciones y el modelo.

**2. Definir la rejilla de hiperparámetros**:

- Usa **ParamGridBuilder** para especificar los valores de los hiperparámetros.

**3. Configurar CrossValidator**:

- Especifica el pipeline, la métrica de evaluación, la rejilla de hiperparámetros y el número de folds.

**4. Entrenar el modelo**:

- Usa **CrossValidator.fit()** para entrenar el modelo con validación cruzada.

**5. Evaluar el modelo**:

- Usa el modelo entrenado para hacer predicciones y evaluar su rendimiento.

### Grid Search y CrossValidator Juntos

**1. ParamGridBuilder** define las combinaciones de hiperparámetros que se probarán.

**2. CrossValidator** toma esa rejilla y:

- Divide los datos en k folds (por ejemplo, 5 folds).

- Para cada combinación de hiperparámetros:

  - Entrena el modelo en k-1 folds.

  - Evalúa el modelo en el fold restante.

  - Repite este proceso k veces (una vez para cada fold).

- Calcula el rendimiento promedio del modelo para cada combinación de hiperparámetros.

**3. Finalmente, CrossValidator** selecciona la combinación de hiperparámetros que produce el mejor rendimiento promedio.

**Ejemplo Completo**

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.ml.Pipeline

// 1. Crear una sesión de Spark
val spark = SparkSession.builder()
  .appName("Grid Search with CrossValidator")
  .master("local[*]")
  .getOrCreate()

// 2. Cargar el dataset winequality-red
val data = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("path/to/winequality-red.csv") // Reemplaza con la ruta correcta

// 3. Preparar los datos: Crear un vector de características
val featureColumns = Array(
  "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
  "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
  "pH", "sulphates", "alcohol"
)

val assembler = new VectorAssembler()
  .setInputCols(featureColumns)
  .setOutputCol("features")

val assembledData = assembler.transform(data)

// 4. Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%)
val Array(trainingData, testData) = assembledData.randomSplit(Array(0.8, 0.2), seed = 42)

// 5. Definir el modelo (Random Forest Regressor)
val rf = new RandomForestRegressor()
  .setLabelCol("quality")
  .setFeaturesCol("features")

// 6. Definir la rejilla de hiperparámetros con ParamGridBuilder
val paramGrid = new ParamGridBuilder()
  .addGrid(rf.maxDepth, Array(5, 10, 15)) // Profundidad máxima del árbol
  .addGrid(rf.numTrees, Array(10, 50, 100)) // Número de árboles
  .build()

// 7. Definir el evaluador (usaremos RMSE)
val evaluator = new RegressionEvaluator()
  .setLabelCol("quality")
  .setPredictionCol("prediction")
  .setMetricName("rmse")

// 8. Crear el Pipeline
val pipeline = new Pipeline()
  .setStages(Array(assembler, rf))

// 9. Configurar CrossValidator
val cv = new CrossValidator()
  .setEstimator(pipeline) // Pipeline con el modelo
  .setEvaluator(evaluator) // Métrica de evaluación
  .setEstimatorParamMaps(paramGrid) // Rejilla de hiperparámetros
  .setNumFolds(5) // Número de folds en la validación cruzada
  .setParallelism(2) // Número de hilos para ejecución en paralelo

// 10. Entrenar el modelo con Cross-Validation
val cvModel = cv.fit(trainingData)

// 11. Obtener el mejor modelo
val bestModel = cvModel.bestModel
println(s"Mejores hiperparámetros: ${bestModel.extractParamMap}")

// 12. Hacer predicciones sobre el conjunto de prueba
val predictions = bestModel.transform(testData)
predictions.select("quality", "prediction").show(10)

// 13. Evaluar el modelo
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE): $rmse")

// 14. Detener la sesión de Spark
spark.stop()

**1. ParamGridBuilder** define una rejilla de hiperparámetros para maxDepth y numTrees.

**2. CrossValidator** prueba cada combinación de hiperparámetros utilizando validación cruzada.

**3. Selecciona** la combinación de hiperparámetros que produce el menor RMSE (Error Cuadrático Medio).

**4. Entrena** el modelo final con los mejores hiperparámetros y lo evalúa en el conjunto de prueba.

### ¿Cuándo es conveniente hacer un análisis completo de hiperparámetros?  

**1. Cuando el rendimiento del modelo es crítico**:

- Si estás trabajando en un problema donde el rendimiento del modelo es crucial (por ejemplo, en aplicaciones médicas, financieras o de seguridad), optimizar los hiperparámetros puede marcar una gran diferencia.

**2. Cuando el modelo tiene muchos hiperparámetros**:

- Algoritmos como Random Forest, Gradient Boosting (por ejemplo, XGBoost o LightGBM), o Redes Neuronales tienen varios hiperparámetros que pueden afectar significativamente el rendimiento. En estos casos, es casi obligatorio realizar una optimización.

**3. Cuando tienes suficientes recursos computacionales**:

- La optimización de hiperparámetros, especialmente con técnicas como Grid Search y Cross-Validation, puede ser costosa en términos de tiempo y recursos. Si tienes acceso a un clúster distribuido o a hardware potente, es más factible realizar un análisis completo.

**4. Cuando el dataset es pequeño o mediano**:

- En datasets pequeños o medianos, la optimización de hiperparámetros es más manejable computacionalmente. En datasets muy grandes, puede ser prohibitivo debido al tiempo y recursos requeridos.

**5. Cuando buscas la mejor generalización**:

- Si tu objetivo es que el modelo generalice bien a datos no vistos, la optimización de hiperparámetros con validación cruzada es una excelente manera de asegurarte de que el modelo no esté sobreajustado o subajustado.

### ¿Cuándo podrías omitir el análisis completo de hiperparámetros?  

**1. Cuando el tiempo o los recursos son limitados**:

- Si estás en una fase exploratoria o prototipando un modelo, podrías usar valores por defecto o ajustar manualmente algunos hiperparámetros clave en lugar de realizar una búsqueda exhaustiva.

**2. Cuando el modelo es simple**:

- Para algoritmos simples como Regresión Lineal o Árboles de Decisión básicos, los hiperparámetros tienen menos impacto en el rendimiento, por lo que podrías omitir una búsqueda exhaustiva.

**3. Cuando el dataset es muy grande**:

- En datasets extremadamente grandes, el costo computacional de la optimización de hiperparámetros puede ser demasiado alto. En estos casos, podrías usar valores por defecto o ajustar manualmente algunos hiperparámetros.

**4. Cuando ya tienes un conocimiento previo**:

- Si ya sabes qué valores de hiperparámetros funcionan bien para tu problema (por ejemplo, por experiencia previa o por literatura), podrías usarlos directamente sin realizar una búsqueda exhaustiva.